In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model

In [2]:
import pandas as pd
import gzip
import json

In [3]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Tools_and_Home_Improvement_5.json.gz')

KeyboardInterrupt: 

In [ ]:
df.head()

In [72]:
def extract_format(style_dict):
    if pd.notna(style_dict) and 'Format:' in style_dict:
        return style_dict['Format:']
    return None

df['style'] = df['style'].apply(extract_format)

In [73]:
df['style'].unique()

array([None, ' Kitchen', ' Electronics', ' Tools & Home Improvement',
       ' Misc. Supplies', ' Home', ' Paperback', ' Tools & Hardware',
       ' Accessory', ' Misc.'], dtype=object)

In [74]:
df = df.drop('image', axis=1)

In [75]:
none_counts = df.isna().sum()

In [76]:
none_counts

overall                 0
verified                0
reviewTime              0
reviewerID              0
asin                    0
style             2064635
reviewerName          149
reviewText            522
summary               279
unixReviewTime          0
vote              1781076
dtype: int64

In [77]:
df = df.dropna()

In [78]:
df['style'].unique()

array([' Electronics', ' Tools & Home Improvement', ' Paperback',
       ' Kitchen', ' Misc. Supplies', ' Home', ' Tools & Hardware',
       ' Accessory', ' Misc.'], dtype=object)

In [79]:
file_path = 'cleaned_review.csv'
df2= pd.read_csv(file_path)

C:\Users\18589\AppData\Local\Temp\ipykernel_21268\3091020896.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df2= pd.read_csv(file_path)


In [80]:
combined_df = pd.concat([df, df2], ignore_index=True)

In [81]:
combined_df.to_csv('cleaned_review.csv')

In [82]:
combined_df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0
0,5.0,True,"08 9, 2016",ARVGPB7UVLOJW,193362275X,Electronics,Dove,"Got this for my husband as a birthday gift, an...",He loves it,1470700800,2,NaN,NaN,NaN,NaN,NaN
1,5.0,True,"08 6, 2016",A291CXNBUMOJRL,193362275X,Electronics,EveryMom,Bought this for my daughter to take to camp an...,Best Book Light I've Had in 25 years!,1470441600,2,NaN,NaN,NaN,NaN,NaN
2,3.0,True,"07 1, 2016",AZ6TCHE726IVP,193362275X,Tools & Home Improvement,alex,"pros\nthe flexible thingy is really good, and ...",Ok,1467331200,2,NaN,NaN,NaN,NaN,NaN
3,5.0,True,"03 4, 2016",A3JGIPEXOZ0ZY6,193362275X,Tools & Home Improvement,vanilllacoke,I've been using this for nearly 2 years for pa...,many fantastic points,1457049600,2,NaN,NaN,NaN,NaN,NaN
4,2.0,True,"08 10, 2013",A30IA5U6ID1QRJ,1933622741,Paperback,Walden,"Okay at first, for about a month, then began t...",little book light,1376092800,3,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623880,5.0,True,"04 3, 2013",AJXV6T2EMQHF0,B004ZBLK0E,Misc.,Shannon H.,"I pretty much love all of the Smash Book line,...",<3,1364947200,2,623178.0,2891.0,1769.0,790.0,443370.0
623881,5.0,True,"03 25, 2015",A19O4K9GZ0RVPH,B0060S43TU,Misc.,Ella,Loved this as well as all other smash things!,Love,1427241600,2,623179.0,2892.0,1770.0,791.0,448229.0
623882,4.0,True,"06 25, 2014",A13LB0Z78L6DSN,B0060S43TU,Misc.,Baker Mom,I purchased the Smash Book (Doodle Red) becaus...,Fun!,1403654400,2,623180.0,2893.0,1771.0,792.0,448231.0
623883,5.0,True,"09 17, 2013",A3JGZLE2L21LWA,B0060S43TU,Misc.,debbi berg,This is a really cute page of different top 10...,SMASH PAD TOP10,1379376000,3,623181.0,2894.0,1772.0,793.0,448232.0


In [3]:
file_path = 'cleaned_review.csv'
combined_df= pd.read_csv(file_path)

C:\Users\18589\AppData\Local\Temp\ipykernel_6428\2632622030.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df= pd.read_csv(file_path)


In [4]:
combined_df = combined_df.loc[:, ~combined_df.columns.str.contains('^Unnamed')]

In [5]:
combined_df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote
0,5.0,True,"08 9, 2016",ARVGPB7UVLOJW,193362275X,Electronics,Dove,"Got this for my husband as a birthday gift, an...",He loves it,1470700800,2
1,5.0,True,"08 6, 2016",A291CXNBUMOJRL,193362275X,Electronics,EveryMom,Bought this for my daughter to take to camp an...,Best Book Light I've Had in 25 years!,1470441600,2
2,3.0,True,"07 1, 2016",AZ6TCHE726IVP,193362275X,Tools & Home Improvement,alex,"pros\nthe flexible thingy is really good, and ...",Ok,1467331200,2
3,5.0,True,"03 4, 2016",A3JGIPEXOZ0ZY6,193362275X,Tools & Home Improvement,vanilllacoke,I've been using this for nearly 2 years for pa...,many fantastic points,1457049600,2
4,2.0,True,"08 10, 2013",A30IA5U6ID1QRJ,1933622741,Paperback,Walden,"Okay at first, for about a month, then began t...",little book light,1376092800,3


In [6]:
df_encoded = pd.get_dummies(combined_df, columns=['style'])

In [7]:
df_encoded.columns

Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'style_ Accessory', 'style_ Amazon Instant Video',
       'style_ Amazon Video', 'style_ Apparel', 'style_ Audio CD',
       'style_ Baby Product', 'style_ Blu-ray', 'style_ CD-ROM',
       'style_ Computer Game', 'style_ DVD', 'style_ DVD Audio',
       'style_ DVD-R', 'style_ DVD-ROM', 'style_ Diary', 'style_ Electronics',
       'style_ Grocery', 'style_ HD DVD', 'style_ Hardcover',
       'style_ Health and Beauty', 'style_ Home', 'style_ Housewares',
       'style_ Interactive DVD', 'style_ Kindle Edition', 'style_ Kitchen',
       'style_ Laser Disc', 'style_ Loose Leaf', 'style_ MP3 Music',
       'style_ Mass Market Paperback', 'style_ Misc.', 'style_ Misc. Supplies',
       'style_ Office Product', 'style_ Paperback', 'style_ Prime Video',
       'style_ Product Bundle', 'style_ Shoes', 'style_ Spiral-bound',
       'style_ Sports',

In [49]:
df_encoded.to_csv('final_data.csv')

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [9]:
df_encoded['verified'] = df_encoded['verified'].astype(int)
df_encoded['vote'] = df_encoded['vote'].replace(',', '', regex=True).fillna(0).astype(int)


In [10]:
X_lr = df_encoded.drop(columns=['reviewTime','reviewerID','asin','reviewerName', 'reviewText', 'summary','overall'])

In [11]:
y = df_encoded['overall']

In [12]:
X_lr

,verified,unixReviewTime,vote,style_ Accessory,style_ Amazon Instant Video,style_ Amazon Video,style_ Apparel,style_ Audio CD,style_ Baby Product,style_ Blu-ray,...,style_ Tools & Hardware,style_ Tools & Home Improvement,style_ Toy,style_ UMD for PSP,style_ Unknown Binding,style_ VHS Tape,style_ Video Game,style_ Videotape,style_ Vinyl,style_ Wireless Phone Accessory
0,1,1470700800,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1470441600,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1467331200,2,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,1457049600,2,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,1,1376092800,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623880,1,1364947200,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
623881,1,1427241600,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
623882,1,1403654400,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
623883,1,1379376000,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:

X_train, X_test, y_train, y_test = train_test_split(X_lr, y, test_size=0.2, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R2 Score:", r2)

Mean Squared Error: 1.8595494507308832
R2 Score: 0.05569713529366038


In [14]:
len(X_test)

124777

In [15]:
threshold = 0.9


correct_predictions = sum(abs(y_test - y_pred) <= threshold)

accuracy_like_metric = correct_predictions / len(y_test)

correct_predictions, accuracy_like_metric



(59793, 0.4791988908212251)

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import make_pipeline


X_train, X_test, y_train, y_test = train_test_split(df_encoded['reviewText'], df_encoded['overall'], test_size=0.2, random_state=42)

model = make_pipeline(TfidfVectorizer(), MultinomialNB())

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.47916683363119805
              precision    recall  f1-score   support

         1.0       0.89      0.07      0.12     15247
         2.0       1.00      0.00      0.00      9611
         3.0       0.44      0.00      0.00     14735
         4.0       0.37      0.00      0.01     26459
         5.0       0.48      1.00      0.64     58725

    accuracy                           0.48    124777
   macro avg       0.64      0.21      0.16    124777
weighted avg       0.54      0.48      0.32    124777



In [42]:
from surprise import Dataset, Reader, SVD,accuracy
from surprise.model_selection import cross_validate, KFold

In [43]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_encoded[['reviewerID', 'asin', 'overall']], reader)

svd = SVD()

cv_results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2027  1.2047  1.2018  1.2023  1.1998  1.2023  0.0016  
MAE (testset)     0.9604  0.9617  0.9606  0.9605  0.9573  0.9601  0.0015  
Fit time          5.62    5.81    6.18    6.00    6.03    5.93    0.19    
Test time         0.49    1.76    0.54    0.54    1.75    1.02    0.61    


In [45]:
import numpy as np

In [46]:

kf = KFold(n_splits=5)
mse_scores = []

for trainset, testset in kf.split(data):
    svd.fit(trainset)

    predictions = svd.test(testset)

    mse = accuracy.mse(predictions, verbose=False)
    mse_scores.append(mse)

average_mse = np.mean(mse_scores)
print(f'Average Test MSE: {average_mse}')

Average Test MSE: 1.4464791511506625


In [48]:
def custom_accuracy(predictions, tolerance=0.5):
    correct = 0
    for uid, iid, true_r, est, _ in predictions:
        if abs(true_r - est) <= tolerance:
            correct += 1
    return correct / len(predictions)

custom_acc = custom_accuracy(predictions)
print(f'Custom Accuracy (with tolerance ±{0.5}): {custom_acc:.2f}')

Custom Accuracy (with tolerance ±0.5): 0.31


In [9]:
from keras.models import load_model

# Replace 'path/to/your/model.h5' with the actual path to your .h5 file
model_path = 'CNN_pred.h5'

# Load the model
loaded_model = load_model(model_path)

# Display the summary of the model
loaded_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 300, 128)          1280000   
                                                                 
 conv1d_3 (Conv1D)           (None, 296, 128)          82048     
                                                                 
 batch_normalization_6 (Batc  (None, 296, 128)         512       
 hNormalization)                                                 
                                                                 
 global_max_pooling1d_3 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 batch_normalization_7 (Batc  (None, 64)              

In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, BatchNormalization

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [17]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df_encoded['reviewText'])
sequences = tokenizer.texts_to_sequences(df_encoded['reviewText'])

In [27]:

maxlen = 300
data = pad_sequences(sequences, maxlen=maxlen)

labels = to_categorical(df_encoded['overall'] - 1)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [28]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("GPU device not found")

Default GPU Device: /device:GPU:0


In [30]:
with tf.device('/GPU:0'):
    model = Sequential()
    model.add(Embedding(10000, 128, input_length=maxlen))  # Embedding layer
    model.add(Conv1D(128, 5, activation='relu'))  # First Convolutional layer
    model.add(BatchNormalization())  # Batch Normalization after Convolutional layer
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64, activation='relu'))  # Additional Dense layer
    model.add(BatchNormalization())  # Batch Normalization after Dense layer
    model.add(Dropout(0.5))  # Dropout for regularization

    model.add(Dense(32, activation='relu'))  # Another Dense layer
    model.add(BatchNormalization())  # Batch Normalization
    model.add(Dropout(0.5))  # Dropout

    model.add(Dense(5, activation='softmax')) 
    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=4, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/4
15598/15598 [==============================] - 107s 7ms/step - loss: 1.0319 - accuracy: 0.5843 - val_loss: 0.8782 - val_accuracy: 0.6412
Epoch 2/4
15598/15598 [==============================] - 111s 7ms/step - loss: 0.9042 - accuracy: 0.6342 - val_loss: 0.8516 - val_accuracy: 0.6508
Epoch 3/4
15598/15598 [==============================] - 111s 7ms/step - loss: 0.8469 - accuracy: 0.6587 - val_loss: 0.8406 - val_accuracy: 0.6569
Epoch 4/4
15598/15598 [==============================] - 112s 7ms/step - loss: 0.7889 - accuracy: 0.6834 - val_loss: 0.8522 - val_accuracy: 0.6551


In [31]:

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')

3900/3900 [==============================] - 10s 2ms/step - loss: 0.8522 - accuracy: 0.6551
Accuracy: 0.65513676404953


In [50]:
model.save('CNN_pred.h5')

In [51]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences



In [56]:
maxlen = 300 
embedding_dim = 128 
num_classes = 5  

with tf.device('/GPU:0'):
    model = Sequential()
    model.add(Embedding(10000, embedding_dim, input_length=maxlen))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(32)) t
    model.add(Dense(64, activation='relu')) 
    model.add(BatchNormalization())
    model.add(Dropout(0.5)) 
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=4, batch_size=32, validation_data=(X_test, y_test))
mse = model.evaluate(X_test, y_test)
print(f'Test MSE: {mse}')

Epoch 1/4
15598/15598 [==============================] - 564s 36ms/step - loss: 0.9804 - accuracy: 0.6018 - val_loss: 0.8659 - val_accuracy: 0.6386
Epoch 2/4
15598/15598 [==============================] - 558s 36ms/step - loss: 0.8603 - accuracy: 0.6521 - val_loss: 0.8149 - val_accuracy: 0.6650
Epoch 3/4
15598/15598 [==============================] - 559s 36ms/step - loss: 0.7990 - accuracy: 0.6766 - val_loss: 0.8012 - val_accuracy: 0.6732
Epoch 4/4
3900/3900 [==============================] - 62s 16ms/step - loss: 0.8092 - accuracy: 0.6695
Test MSE: [0.8092278242111206, 0.6694823503494263]


In [57]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')

3900/3900 [==============================] - 60s 15ms/step - loss: 0.8092 - accuracy: 0.6695
Accuracy: 0.6694823503494263


In [58]:
# Predicting the values for the test set
y_pred_rnn = model.predict(X_test)

# If your target variable (y_test) is one-hot encoded, convert predictions back to labels
y_pred_rnn = np.argmax(y_pred_rnn, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Calculate the MSE
mse_rnn = mean_squared_error(y_test_labels, y_pred_rnn)
print(f"RNN Model MSE: {mse_rnn}")

3900/3900 [==============================] - 57s 14ms/step
RNN Model MSE: 0.7974867163018825


In [59]:
model.save('RNN_pred.h5')

In [1]:
import matplotlib as plt
import pandas as pd

In [2]:
data = {
    "Model": ['Linear regression', 'Multinomial NB', 'SVD','Latent Factor', 'CNN', 'LSTM'],
    "Accuracy": ['NA', '48.7%', 'NA', 'NA', '65.5%', '66.9%']
}

df = pd.DataFrame(data)

In [3]:
from IPython.display import display, HTML

html_table = df.to_html(index=False)
display(HTML(html_table))

Model,Accuracy
Linear regression,NA
Multinomial NB,48.7%
SVD,NA
Latent Factor,NA
CNN,65.5%
LSTM,66.9%


In [4]:
data = {
    "Model": ['Linear regression', 'Multinomial NB', 'SVD', 'Latent Factor', 'CNN', 'LSTM'],
    "MSE": ['1.86', '1.13', '1.45','0.864', '0.79', '0.74']
}
df = pd.DataFrame(data)
html_table = df.to_html(index=False)
display(HTML(html_table))

Model,MSE
Linear regression,1.86
Multinomial NB,1.13
SVD,1.45
Latent Factor,0.864
CNN,0.79
LSTM,0.74
